## Data preparation and design of LSTM - based Timeseries models and variants
This notebook presents the process of preparing input data and constructing various LSTM-based model variants to capture temporal patterns in apparent temperature.

### Main idea:
The model is built for a multivariate time series forecasting task.

Input: Preprocessed apparent temperature data with 6 descriptive features, including day, month, year, air temperature, dew point, and relative humidity.

Output: 2 target features such as mean apparent temperature, max apparent temperature.


In [11]:
import typing
from itertools import islice
import pandas as pd
import numpy as np

import sklearn
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import keras
from keras import layers, ops

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers

from math import radians, sin, cos, sqrt, atan2

import matplotlib.pyplot as plt

In [12]:
df_CaMau = pd.read_csv('/kaggle/input/at-vietnam-data/Data_AT_FilteredDate/CaMau_FilteredDate.csv')
df_LangSon = pd.read_csv('/kaggle/input/at-vietnam-data/Data_AT_FilteredDate/LangSon_FilteredDate.csv')
df_LaoCai = pd.read_csv('/kaggle/input/at-vietnam-data/Data_AT_FilteredDate/LaoCai_FilteredDate.csv')
df_NoiBai = pd.read_csv('/kaggle/input/at-vietnam-data/Data_AT_FilteredDate/NoiBai_FilteredDate.csv')
df_PhuBai = pd.read_csv('/kaggle/input/at-vietnam-data/Data_AT_FilteredDate/PhuBai_FilteredDate.csv')
df_QuyNhon = pd.read_csv('/kaggle/input/at-vietnam-data/Data_AT_FilteredDate/QuyNhon_FilteredDate.csv')
df_TPHCM = pd.read_csv('/kaggle/input/at-vietnam-data/Data_AT_FilteredDate/TPHCM_FilteredDate.csv')
df_Vinh = pd.read_csv('/kaggle/input/at-vietnam-data/Data_AT_FilteredDate/Vinh_FilteredDate.csv')

station_dfs = {
    'Noi_Bai': df_NoiBai,
    'Lang_Son': df_LangSon,
    'Lao_Cai': df_LaoCai,

    'Vinh': df_Vinh,
    'Phu_Bai': df_PhuBai,
    'Quy_Nhon': df_QuyNhon,

    'TPHCM': df_TPHCM,
    'Ca_Mau': df_CaMau
}

### Data preparation

Data preparation steps:
1. **Normalize data to range [0, 1] with MinMaxScaler**: Function scale_data_per_station
   
2. **Split dataset into X and y set**: The dataset is split into input features (X) and target variables (y), with X including descriptive features and y representing the target apparent temperature values to be predicted. Function: split_X_y

   
3. **Divide X and y into three sets (training, validation, and testing)**: Based on the 80 - 10 - 10 ratio, 80% of the data is used for training, 10% for validation, and 10% for testing. Function: split_train_val_test_set

4. **Generate sliding windows for time series modeling**: Function create_tf_dataset


In [13]:
# Normalize data
def scale_data_per_station(station_data_dict, features, feature_range=(0, 1),
                           handle_outliers=True, scaling_method='minmax'):
    scaled_data_dict = {}
    scaler_dict = {}
    
    for station_name, df in station_data_dict.items():
        df_scaled = df.copy()
        feature_scalers = {}
        
        for feature in features:
            feature_data = df_scaled[feature].copy()
            
            if handle_outliers:
                Q1 = feature_data.quantile(0.25)  
                Q3 = feature_data.quantile(0.75)  
                IQR = Q3 - Q1
                lower_bound = Q1 - 1.5 * IQR
                upper_bound = Q3 + 1.5 * IQR
                
                feature_data = feature_data.clip(lower_bound, upper_bound)
            
            feature_array = feature_data.values.reshape(-1, 1)
            
            # Chọn scaler phù hợp
            if scaling_method == 'robust':
                scaler = RobustScaler()
                scaled_values = scaler.fit_transform(feature_array)
            else:  # 'minmax'
                scaler = MinMaxScaler(feature_range=feature_range)
                scaled_values = scaler.fit_transform(feature_array)
            
            # Cập nhật DataFrame
            df_scaled[feature] = scaled_values
            feature_scalers[feature] = scaler
            
        scaled_data_dict[station_name] = df_scaled
        scaler_dict[station_name] = feature_scalers
        
    return scaled_data_dict, scaler_dict


features_to_scale = ['YEAR', 'MONTH', 'DAY', 'DEW_2', 'TMP_2', 'RH', 'AT mean', 'AT max']

scaled_data, scalers = scale_data_per_station(
    station_dfs, 
    features_to_scale,
    handle_outliers=False,
    scaling_method='minmax' 
)

In [14]:
# Tách đặc trưng mô tả (X) và mục tiêu (y)
def split_X_y(df, X_features, y_features):
    X_groups = df[X_features]
    y_groups = df[y_features]
    return np.array(X_groups), np.array(y_groups)
    
X_features = ['YEAR', 'MONTH', 'DAY', 'DEW_2', 'TMP_2', 'RH']
y_features = ['AT mean', 'AT max']
X, y = split_X_y(scaled_data['Ca_Mau'], X_features, y_features)

In [15]:
# Tách tập dữ liệu theo tỷ lệ 80 - 10 - 10% 
def split_train_val_test_set(X, y, train_size=0.8, val_size=0.1):
    num_timestamp = len(X)
    num_train, num_val = (
        int(num_timestamp * train_size),
        int(num_timestamp * val_size)
    )

    X_train = X[:num_train]
    y_train = y[:num_train]
    X_val = X[num_train: (num_train + num_val)]
    y_val = y[num_train: (num_train + num_val)]
    X_test = X[(num_train + num_val):]
    y_test = y[(num_train + num_val):]

    return (X_train, y_train), (X_val, y_val), (X_test, y_test)

(X_train, y_train), (X_val, y_val), (X_test, y_test) = split_train_val_test_set(X, y)
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (9408, 6)
y_train shape: (9408, 2)
X_val shape: (1176, 6)
y_val shape: (1176, 2)
X_test shape: (1177, 6)
y_test shape: (1177, 2)


In [16]:
# Tạo cửa sổ dữ liệu
# Trạm miền Bắc + Trung: batch_size = 64
# Trạm miền Nam: batch_size = 8

batch_size = 8
input_sequence_length = 31
forecast_horizon = 1
multi_horizon = False

def create_tf_dataset(
        X_full: np.ndarray,
        y_full: np.ndarray,
        input_sequence_length: int = 30,
        forecast_horizon: int = 1,
        batch_size: int = 64,
        shuffle: bool = True,
        multi_horizon: bool = False,
):
    inputs = keras.utils.timeseries_dataset_from_array(
        X_full[:-forecast_horizon], 
        None,
        sequence_length=input_sequence_length,
        shuffle=False,
        batch_size=batch_size
    )

    target_offset = (
        input_sequence_length
        if multi_horizon
        else input_sequence_length + forecast_horizon - 1
    )

    target_seq_length = forecast_horizon if multi_horizon else 1

    targets = keras.utils.timeseries_dataset_from_array(
        y_full[target_offset:],
        None,
        sequence_length=target_seq_length,
        shuffle=False,
        batch_size=batch_size
    )

    dataset = tf.data.Dataset.zip((inputs, targets))

    if shuffle:
        dataset = dataset.shuffle(100)

    return dataset.prefetch(16).cache()


train_dataset = create_tf_dataset(
    X_train, 
    y_train,
    input_sequence_length=input_sequence_length,
    forecast_horizon=forecast_horizon,
    batch_size=batch_size,
    shuffle=True,
    multi_horizon=multi_horizon
)

val_dataset = create_tf_dataset(
    X_val, 
    y_val,
    input_sequence_length=input_sequence_length,
    forecast_horizon=forecast_horizon,
    batch_size=batch_size,
    shuffle=True,
    multi_horizon=multi_horizon
)

test_dataset = create_tf_dataset(
    X_test, 
    y_test,
    input_sequence_length=input_sequence_length,
    forecast_horizon=forecast_horizon,
    batch_size=len(X_test),  # Sử dụng toàn bộ test set
    shuffle=False,
    multi_horizon=multi_horizon
)

print(train_dataset)
print(val_dataset)
print(test_dataset)

2025-07-11 08:30:41.941137: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


<CacheDataset element_spec=(TensorSpec(shape=(None, None, 6), dtype=tf.float64, name=None), TensorSpec(shape=(None, None, 2), dtype=tf.float64, name=None))>
<CacheDataset element_spec=(TensorSpec(shape=(None, None, 6), dtype=tf.float64, name=None), TensorSpec(shape=(None, None, 2), dtype=tf.float64, name=None))>
<CacheDataset element_spec=(TensorSpec(shape=(None, None, 6), dtype=tf.float64, name=None), TensorSpec(shape=(None, None, 2), dtype=tf.float64, name=None))>


### Building LSTM and BiLSTM models

In [17]:
class LSTM(layers.Layer):
    def __init__(
        self,
        lstm_units: int,
        input_seq_len: int,
        in_feat: int,
        forecast_horizon: int,
        **kwargs,
    ):
        super().__init__(**kwargs)


        self.lstm1 = layers.LSTM(
            lstm_units, 
            activation="tanh",
            return_sequences=True,
        )

        self.lstm2 = layers.LSTM(
            lstm_units // 2,
            activation="tanh", 
            return_sequences=True
        )


        self.lstm = layers.LSTM(
            lstm_units // 4,
            activation="tanh", 
            return_sequences=False
        )

        self.dense_32 = layers.Dense(32)
        self.dense = layers.Dense(forecast_horizon * 2)

        self.dropout_02 = layers.Dropout(0.02)
        self.dropout_01 = layers.Dropout(0.1)
        
        self.input_seq_len = input_seq_len
        self.in_feat = in_feat
        self.forecast_horizon = forecast_horizon

        self.norm = layers.LayerNormalization()

    def call(self, inputs):
        print("Input: ", inputs.shape)
        
        lstm1 = self.lstm1(inputs)
        lstm1 = self.dropout_02(lstm1)
        print("LSTM 1: ", lstm1.shape)

        lstm2 = self.lstm2(lstm1)
        lstm2 = self.dropout_02(lstm2)
        print("LSTM 2: ", lstm2.shape)
        
        lstm_out = self.lstm(lstm2)
        lstm_out = self.dropout_02(lstm_out)
        print("LSTM out: ", lstm_out.shape)

        dense1 = self.dense_32(lstm_out)
        dense1 = self.dropout_02(dense1)
        print("Dense 32 shape: ", dense1.shape)
        
        dense_output = self.dense(dense1)
        print("Dense output shape: ", dense_output.shape)
        
        batch_size = tf.shape(dense_output)[0]  
        output = tf.reshape(dense_output, (batch_size, self.forecast_horizon, 2))
        
        return output

In [18]:
class BiLSTM(layers.Layer):
    def __init__(
        self, 
        lstm_units: int,
        input_seq_len: int,
        in_feat: int,
        forecast_horizon: int,
        **kwargs,
    ):
        super().__init__(**kwargs)
        
        initializer = tf.keras.initializers.RandomUniform(minval=-0.08, maxval=0.08)

        self.bilstm1 = layers.Bidirectional(
            layers.LSTM(
                lstm_units,
                activation="tanh",
                return_sequences=True,
            ),
            merge_mode='ave'
        )

        self.bilstm2 = layers.Bidirectional(
            layers.LSTM(
                lstm_units // 2,
                activation="tanh",
                return_sequences=True
            ),
            merge_mode='ave'
        )

        self.bilstm3 = layers.Bidirectional(
            layers.LSTM(
                lstm_units // 4,
                activation="tanh",
                return_sequences=False
            ),
            merge_mode='ave'
        )

        self.dense_32 = layers.Dense(32)
        self.dense = layers.Dense(2)

        self.dropout_02 = layers.Dropout(0.2)
        self.dropout_01 = layers.Dropout(0.02) #0.01       
        
        self.input_seq_len = input_seq_len
        self.in_feat = in_feat
        self.forecast_horizon = forecast_horizon


    def call(self, inputs):
        bilstm1 = self.bilstm1(inputs)
        bilstm1 = self.dropout_02(bilstm1)
        
        bilstm2 = self.bilstm2(bilstm1)
        bilstm2 = self.dropout_02(bilstm2)
        
        bilstm_out = self.bilstm3(bilstm2)
        bilstm_out = self.dropout_02(bilstm_out)

        dense1 = self.dense_32(bilstm_out)
        dense1 = self.dropout_01(dense1)
        
        dense_output = self.dense(dense1)
        # dense_output = self.dropout_01(dense_output)
    
        batch_size = tf.shape(dense_output)[0]  
        output = tf.reshape(dense_output, (batch_size, self.forecast_horizon, 2))
        
        return output

In [19]:
in_feat = 6
epochs = 1000
lstm_units = 256
forecast_horizon = 1


lstm_predictor = LSTM(
    lstm_units=lstm_units,
    input_seq_len=input_sequence_length,
    in_feat = in_feat,
    forecast_horizon = forecast_horizon
)

bilstm_predictor = BiLSTM(
    lstm_units=lstm_units,
    input_seq_len=input_sequence_length,
    in_feat = in_feat,
    forecast_horizon = forecast_horizon
)

inputs = layers.Input(shape=(input_sequence_length, in_feat))
outputs = bilstm_predictor(inputs)

optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.0001,
    clipnorm=1.0,
)
model = keras.models.Model(inputs, outputs)
model.compile(
    loss='mean_squared_error',
    optimizer=optimizer,
    metrics=["mean_squared_error"]
)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 31, 6)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bi_lstm (BiLSTM)                │ (None, 1, 2)           │     1,033,826 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,033,826 (3.94 MB)

 Trainable params: 1,033,826 (3.94 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import time
from tensorflow.keras.callbacks import EarlyStopping

# Thiết lập Early Stopping
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=5,  
    restore_best_weights=True
)

start_time = time.time()

# Huấn luyện mô hình
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=1000,
    callbacks=[early_stopping],
)

end_time = time.time()
training_time = end_time - start_time
print(training_time)

# Chuyển đổi sang định dạng giờ:phút:giây
hours, remainder = divmod(training_time, 3600)
minutes, seconds = divmod(remainder, 60)

print(f'Thời gian huấn luyện: {int(hours)} giờ {int(minutes)} phút {seconds:.2f} giây')
print(f'Tổng số epoch đã huấn luyện: {len(history.history["loss"])}')
print(f'Giá trị loss cuối cùng: {history.history["loss"][-1]:.4f}')
print(f'Giá trị validation loss cuối cùng: {history.history["val_loss"][-1]:.4f}')

# Tính thời gian trung bình cho mỗi epoch
avg_time_per_epoch = training_time / len(history.history["loss"])
print(f'Thời gian trung bình cho mỗi epoch: {avg_time_per_epoch:.2f} giây')

Epoch 1/1000
 162/1173 ━━━━━━━━━━━━━━━━━━━━ 1:55 114ms/step - loss: 0.0470 - mean_squared_error: 0.0470

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Loss (huấn luyện)')
plt.plot(history.history['val_loss'], label='Loss (xác thực)')
plt.xlabel('Epoch', fontsize=12, fontfamily='Times New Roman')
plt.ylabel('Giá trị loss', fontsize=12, fontfamily='Times New Roman')
plt.title('GIÁ TRỊ LOSS TRONG QUÁ TRÌNH HUẤN LUYỆN VÀ XÁC THỰC', fontsize=14, fontfamily='Times New Roman', fontweight='bold')
plt.legend()
plt.show()

In [ ]:
x_test, y = next(test_dataset.as_numpy_iterator())
y_pred = model.predict(x_test)

print("x test shape: ", x_test.shape)
print("y shape: ", y.shape)
print("y pred shape: ", y_pred.shape)

In [ ]:
x_train, y_train = next(train_dataset.as_numpy_iterator())
print("x train shape: ", x_train.shape)
print("y train shape: ", y_train.shape)

x_val, y_val = next(val_dataset.as_numpy_iterator())
print("x val shape: ", x_val.shape)
print("y val shape: ", y_val.shape)

In [ ]:
# Đếm số batch trong train_dataset
train_batches = 0
for _ in train_dataset:
    train_batches += 1
print(f"Số lượng batch trong train_dataset: {train_batches}")

# Đếm số batch trong val_dataset
val_batches = 0
for _ in val_dataset:
    val_batches += 1
print(f"Số lượng batch trong val_dataset: {val_batches}")

In [ ]:
# Đếm chính xác số mẫu trong train_dataset
train_samples = 0
for x_batch, _ in train_dataset:
    train_samples += x_batch.shape[0]
print(f"Số lượng mẫu thực tế trong train_dataset: {train_samples}")

# Đếm chính xác số mẫu trong val_dataset
val_samples = 0
for x_batch, _ in val_dataset:
    val_samples += x_batch.shape[0]
print(f"Số lượng mẫu thực tế trong val_dataset: {val_samples}")

In [ ]:
def evaluate_all(y_true, y_pred):
    y_test_flat = y_true.reshape(-1, y_true.shape[-1]) 
    y_pred_flat = y_pred.reshape(-1, y_pred.shape[-1])
    
    r2 = r2_score(y_test_flat, y_pred_flat)
    mse = mean_squared_error(y_test_flat, y_pred_flat)
    rmse = np.sqrt(mean_squared_error(y_test_flat, y_pred_flat))
    mae = mean_absolute_error(y_test_flat, y_pred_flat)
    
    print(f"R²: {r2 * 100:.4f}%")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")

evaluate_all(y, y_pred) 

In [ ]:
def evaluate_all_by_target_features(y_true, y_pred):
    y_true_flat = y_true.reshape(-1, y_true.shape[-1]) 
    y_pred_flat = y_pred.reshape(-1, y_pred.shape[-1])

    for i in range(y_true_flat.shape[1]):
        y_true_feature = y_true_flat[:, i]
        y_pred_feature = y_pred_flat[:, i]
        
        r2 = r2_score(y_true_feature, y_pred_feature)
        mse = mean_squared_error(y_true_feature, y_pred_feature)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_true_feature, y_pred_feature)
        
        print(f"Feature {i + 1}:")
        print(f"  R²: {r2 * 100:.4f}%")
        print(f"  MSE: {mse:.4f}")
        print(f"  RMSE: {rmse:.4f}")
        print(f"  MAE: {mae:.4f}")
        print("-" * 30)

evaluate_all_by_target_features(y, y_pred)

In [ ]:
target_features=['AT mean', 'AT max']
station = 'Ca Mau'
y_true_flat = y.reshape(-1, y.shape[-1])
y_pred_flat = y_pred.reshape(-1, y_pred.shape[-1])
    
# Khởi tạo mảng để lưu dữ liệu đã inverse transform
y_true_original = np.copy(y_true_flat)
y_pred_original = np.copy(y_pred_flat)
    
# Scale ngược dữ liệu nếu scalers được cung cấp
if scalers is not None and station in scalers:
    station_scalers = scalers[station]
        
    for i, feature in enumerate(target_features):
        if feature in station_scalers:
            feature_scaler = station_scalers[feature]
                
            # Inverse transform từng cột riêng biệt
            y_true_feature = y_true_flat[:, i].reshape(-1, 1)
            y_pred_feature = y_pred_flat[:, i].reshape(-1, 1)
                
            y_true_original[:, i:i+1] = feature_scaler.inverse_transform(y_true_feature)
            y_pred_original[:, i:i+1] = feature_scaler.inverse_transform(y_pred_feature)

for i, feature in enumerate(target_features):
    y_true_feature = y_true_original[:, i]
    y_pred_feature = y_pred_original[:, i]
        
    r2 = r2_score(y_true_feature, y_pred_feature)
    mse = mean_squared_error(y_true_feature, y_pred_feature)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true_feature, y_pred_feature)
        
    print(f"{feature}:")
    print(f"  Hệ số xác định (R²): {r2 * 100:.3f}%")
    print(f"  Sai số bình phương trung bình (MSE): {mse:.3f}")
    print(f"  Căn sai số bình phương trung bình (RMSE): {rmse:.3f}")
    print(f"  Sai số tuyệt đối trung bình (MAE): {mae:.3f}")
    print("-" * 50)